In [1]:
import time
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor

def start_stable_driver():
    options = Options()
    options.page_load_strategy = 'eager' # Sadece metin yapısını yükle, hızı artır
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("detach", True) # Kapanmayı engelle
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    
    # Resim yüklemeyi kapatarak internetten tasarruf et
    prefs = {"profile.managed_default_content_settings.images": 2}
    options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return driver

# Global Ayarlar
BASE_URL = "https://www.sikayetvar.com/arcelik/bulasik-makinesi"
detail_links = [] # Link havuzu
all_complaints = [] # Veri havuzu

In [2]:
# Eğer daha önce link topladıysan sakın Kernel'ı restart etme, liste havuzda kalsın.
if 'detail_links' not in locals():
    detail_links = []

# Başlangıç Ayarları
target_pages = 200
current_page = 1  # Kaldığın sayfa numarasını buradan güncelle
refresh_interval = 25  # Her 25 sayfada bir tarayıcıyı kapatıp açacak

def run_scraper(start_p, end_p):
    global detail_links
    driver = start_stable_driver()
    
    print(f"🌐 Tarayıcı başlatıldı. {start_p} ile {end_p} arası sayfalar taranıyor...")
    
    for p in range(start_p, end_p + 1):
        success = False
        attempt = 0
        while not success and attempt < 3:
            try:
                driver.get(f"{BASE_URL}?page={p}")
                time.sleep(2.5) # Sayfa yüklenme payı
                
                # Linkleri topla
                links = driver.find_elements(By.CSS_SELECTOR, "a.complaint-layer")
                
                if len(links) > 0:
                    for l in links:
                        href = l.get_attribute("href")
                        if href and "/arcelik/" in href:
                            if not any(d['url'] == href for d in detail_links):
                                detail_links.append({"baslik": l.get_attribute("title"), "url": href})
                    
                    if p % 5 == 0:
                        print(f"✅ Sayfa {p} bitti. Havuz: {len(detail_links)} link.")
                    success = True
                else:
                    print(f"🛑 Sayfa {p} boş geldi! Manuel doğrulama gerekebilir.")
                    time.sleep(5)
                    attempt += 1
            except Exception as e:
                print(f"⚠️ Hata (Sayfa {p}): {str(e)[:50]}")
                attempt += 1
                time.sleep(2)
    
    driver.quit()
    print("🧹 Tarayıcı şişmeyi önlemek için kapatıldı.")

# Ana Döngü: Parçalar halinde çalıştır
while current_page <= target_pages:
    next_stop = min(current_page + refresh_interval - 1, target_pages)
    run_scraper(current_page, next_stop)
    current_page = next_stop + 1
    if current_page <= target_pages:
        print(f"🔄 Dinleniliyor ve sıradaki {refresh_interval} sayfa için hazırlanılıyor...")
        time.sleep(5)

print(f"🏁 TÜM İŞLEM BİTTİ! Toplam: {len(detail_links)} link toplandı.")

🌐 Tarayıcı başlatıldı. 1 ile 25 arası sayfalar taranıyor...
✅ Sayfa 5 bitti. Havuz: 116 link.
✅ Sayfa 10 bitti. Havuz: 234 link.
✅ Sayfa 15 bitti. Havuz: 352 link.
✅ Sayfa 20 bitti. Havuz: 470 link.
✅ Sayfa 25 bitti. Havuz: 585 link.
🧹 Tarayıcı şişmeyi önlemek için kapatıldı.
🔄 Dinleniliyor ve sıradaki 25 sayfa için hazırlanılıyor...
🌐 Tarayıcı başlatıldı. 26 ile 50 arası sayfalar taranıyor...
✅ Sayfa 30 bitti. Havuz: 700 link.
✅ Sayfa 35 bitti. Havuz: 813 link.
✅ Sayfa 40 bitti. Havuz: 926 link.
✅ Sayfa 45 bitti. Havuz: 1030 link.
✅ Sayfa 50 bitti. Havuz: 1104 link.
🧹 Tarayıcı şişmeyi önlemek için kapatıldı.
🔄 Dinleniliyor ve sıradaki 25 sayfa için hazırlanılıyor...
🌐 Tarayıcı başlatıldı. 51 ile 75 arası sayfalar taranıyor...
✅ Sayfa 55 bitti. Havuz: 1174 link.
✅ Sayfa 60 bitti. Havuz: 1235 link.
✅ Sayfa 65 bitti. Havuz: 1295 link.
✅ Sayfa 70 bitti. Havuz: 1353 link.
✅ Sayfa 75 bitti. Havuz: 1410 link.
🧹 Tarayıcı şişmeyi önlemek için kapatıldı.
🔄 Dinleniliyor ve sıradaki 25 sayfa için

In [3]:
# Bağlantıları hızlandırmak için session yapılandırması
session = requests.Session()
adapter = requests.adapters.HTTPAdapter(pool_connections=50, pool_maxsize=50)
session.mount('https://', adapter)

def fetch_detail_final(item):
    url = item['url']
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    }
    
    # Hata oranını düşürmek için 4 deneme (retry) hakkı
    for attempt in range(4):
        try:
            # Her denemede bekleme süresini biraz artırarak siteyi yormadan dene
            time.sleep(random.uniform(0.2, 0.4) * (attempt + 1))
            
            res = session.get(url, headers=headers, timeout=10)
            
            if res.status_code == 200:
                soup = BeautifulSoup(res.text, 'html.parser')
                
                # İçerik toplama (Start/End Mantığı)
                start_node = soup.find("div", {"data-ga-element": "Complaint_Content_Start"})
                content_parts = []
                
                if start_node:
                    for sibling in start_node.find_next_siblings():
                        if sibling.name == "div" and sibling.get("data-ga-element") == "Complaint_Content_End":
                            break
                        if sibling.name == "p":
                            content_parts.append(sibling.get_text(strip=True))
                
                full_text = " ".join(content_parts)
                
                # Yedek Yöntem: Paragraflar yoksa ana kutu
                if not full_text:
                    desc_div = soup.find("div", class_="complaint-detail-description")
                    full_text = desc_div.get_text(strip=True) if desc_div else ""

                if len(full_text) > 5:
                    return {"baslik": item["baslik"], "url": url, "aciklama": full_text}
            
            elif res.status_code == 429: # Çok fazla istek uyarısı
                time.sleep(5)
        except:
            continue
            
    return {"baslik": item["baslik"], "url": url, "aciklama": "HATA: Alınamadı."}

print(f"🚀 {len(detail_links)} link işleniyor (Bulaşık Makinesi)...")
all_complaints = []
start_time = time.time()

# max_workers=12: Hız ve güvenlik arasındaki en iyi denge
with ThreadPoolExecutor(max_workers=12) as executor:
    futures = [executor.submit(fetch_detail_final, link) for link in detail_links]
    
    for i, future in enumerate(futures, 1):
        all_complaints.append(future.result())
        
        if i % 100 == 0:
            elapsed = time.time() - start_time
            remaining = (elapsed / i) * (len(detail_links) - i) / 60
            print(f"📊 İlerleme: {i}/{len(detail_links)} | Kalan Tahmini: ~{remaining:.1f} dk")



🚀 2183 link işleniyor (Bulaşık Makinesi)...
📊 İlerleme: 100/2183 | Kalan Tahmini: ~35.4 dk
📊 İlerleme: 200/2183 | Kalan Tahmini: ~36.6 dk
📊 İlerleme: 300/2183 | Kalan Tahmini: ~29.6 dk
📊 İlerleme: 400/2183 | Kalan Tahmini: ~25.0 dk
📊 İlerleme: 500/2183 | Kalan Tahmini: ~23.5 dk
📊 İlerleme: 600/2183 | Kalan Tahmini: ~23.2 dk
📊 İlerleme: 700/2183 | Kalan Tahmini: ~21.2 dk
📊 İlerleme: 800/2183 | Kalan Tahmini: ~19.3 dk
📊 İlerleme: 900/2183 | Kalan Tahmini: ~18.0 dk
📊 İlerleme: 1000/2183 | Kalan Tahmini: ~15.8 dk
📊 İlerleme: 1100/2183 | Kalan Tahmini: ~15.0 dk
📊 İlerleme: 1200/2183 | Kalan Tahmini: ~13.2 dk
📊 İlerleme: 1300/2183 | Kalan Tahmini: ~12.1 dk
📊 İlerleme: 1400/2183 | Kalan Tahmini: ~11.0 dk
📊 İlerleme: 1500/2183 | Kalan Tahmini: ~9.7 dk
📊 İlerleme: 1600/2183 | Kalan Tahmini: ~8.1 dk
📊 İlerleme: 1700/2183 | Kalan Tahmini: ~6.8 dk
📊 İlerleme: 1800/2183 | Kalan Tahmini: ~5.4 dk
📊 İlerleme: 1900/2183 | Kalan Tahmini: ~5.7 dk
📊 İlerleme: 2000/2183 | Kalan Tahmini: ~3.6 dk
📊 İlerleme:

In [4]:
# Final Kayıt
df_bulasik = pd.DataFrame(all_complaints)
df_bulasik.to_excel("arcelik_bulasik_makinesi_final.xlsx", index=False)
print(f"✅ İŞLEM TAMAMLANDI! Dosya: arcelik_bulasik_makinesi_final.xlsx")

✅ İŞLEM TAMAMLANDI! Dosya: arcelik_bulasik_makinesi_final.xlsx
